In [27]:
#import Library Numpy, Pandas
import pandas as pd
import numpy as np

#Library Tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# import library plotly,itertools
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from itertools import cycle

In [28]:
#read Data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data TPA Banyuurip 2019-2022.xlsx')
#Ambil data sampah yang bukan sampah terpilah
data = df[df.Barang != 'Sampah Terpilah']
#resample data sampah perhari
data_1 = data.set_index('Tanggal').resample('D')["Netto (kg)"].sum().loc[lambda x: x!= 0].reset_index()
#ubah volume sampah menjadi ton
data_1['Netto (kg)'] = data_1['Netto (kg)']/1000
data_1

,Tanggal,Netto (kg)
0,2019-01-03,21.530
1,2019-01-04,52.414
2,2019-01-05,59.410
3,2019-01-06,52.080
4,2019-01-07,85.572
...,...,...
1408,2022-12-27,78.609
1409,2022-12-28,70.760
1410,2022-12-29,93.859
1411,2022-12-30,75.554


In [29]:
# Filter data between two dates
filtered_df_2020 = data_1.loc[(data_1['Tanggal'] >= '2022-01-01')
                     & (data_1['Tanggal'] < '2022-01-31')]
filtered_df_2020

,Tanggal,Netto (kg)
1051,2022-01-01,48.520
1052,2022-01-02,26.630
1053,2022-01-03,82.280
1054,2022-01-04,79.210
1055,2022-01-05,67.975
1056,2022-01-06,65.280
1057,2022-01-07,53.010
1058,2022-01-08,30.635
1059,2022-01-09,21.090
1060,2022-01-10,85.245


In [30]:
#Visualisasi Data Sampah Perhari
fig = px.line(data_1, x="Tanggal", y="Netto (kg)", height=600,labels={'Netto (kg)':'Volume Sampah(ton)'}) 
fig.update_layout(title_text='Volume Sampah Harian Pada TPSA Banyuurip 2019-2022',
                plot_bgcolor='white', font_size=15, font_color='black')
fig.show()

In [31]:
#Data Describe Volume Sampah
data_1.describe()

,Netto (kg)
count,1413.000000
mean,65.872577
std,20.925057
min,1.220000
25%,57.365000
50%,70.085000
75%,79.460000
max,117.274000


In [32]:
#Normalisasi Data
from sklearn.preprocessing import MinMaxScaler
close_stock = data_1.copy()
del data_1['Tanggal']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(data_1).reshape(-1,1))
print(closedf.shape)

(1413, 1)


In [33]:
#Cek Data Normalisasi
datanormalisasi= pd.DataFrame(closedf)
datanormalisasi.head(20)


,0
0,0.175005
1,0.441122
2,0.501405
3,0.438244
4,0.726834
5,0.979906
6,0.848071
7,0.868234
8,0.544057
9,0.734572


In [34]:
#Sliding Window
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----29 30
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
    
time_step = 30

In [35]:
#split Data training dan data test
training_size=int(len(closedf)*0.80)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]

In [36]:
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [37]:
# Create Model Backpropagation
model = Sequential() 
model.add(Dense(7, input_dim=time_step, kernel_initializer='normal'))
model.add(Dense(1,kernel_initializer='normal'))
# Compile model
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt)

In [38]:
#input epoch,batch_size
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1000,batch_size=64,verbose=1)

Epoch 1/1000
18/18 [==============================] - 1s 15ms/step - loss: 0.0973 - val_loss: 0.0352
Epoch 2/1000
18/18 [==============================] - 0s 6ms/step - loss: 0.0338 - val_loss: 0.0242
Epoch 3/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0300 - val_loss: 0.0224
Epoch 4/1000
18/18 [==============================] - 0s 8ms/step - loss: 0.0291 - val_loss: 0.0226
Epoch 5/1000
18/18 [==============================] - 0s 6ms/step - loss: 0.0278 - val_loss: 0.0206
Epoch 6/1000
18/18 [==============================] - 0s 6ms/step - loss: 0.0258 - val_loss: 0.0198
Epoch 7/1000
18/18 [==============================] - 0s 6ms/step - loss: 0.0243 - val_loss: 0.0195
Epoch 8/1000
18/18 [==============================] - 0s 6ms/step - loss: 0.0234 - val_loss: 0.0190
Epoch 9/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0233 - val_loss: 0.0197
Epoch 10/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0234 - val_loss: 0.019

In [39]:
#model.save('mse_0.18870.h5')

In [40]:
#Hasil Prediksi Dari Model
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

8/8 [==============================] - 0s 2ms/step


((1099, 1), (252, 1))

In [41]:
# Evaluasi Hasil dengan MSE
from sklearn.metrics import mean_squared_error
print("Train data MSE: ", mean_squared_error(y_train,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data MSE: ", mean_squared_error(y_test,test_predict))

Train data MSE:  0.021776883529847423
-------------------------------------------------------------------------------------
Test data MSE:  0.01893948412534921


In [42]:
#Denormalisasi Data

train_predict_original = scaler.inverse_transform(train_predict)
test_predict_original = scaler.inverse_transform(test_predict)

In [43]:
#Plotting Data train Prediksi

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict_original
print("Train predicted data: ", trainPredictPlot.shape)

# Plotting Data Test Prediksi
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict_original
print("Test predicted data: ", testPredictPlot.shape)


#Buat Dataframe Dari Hasil Prediksi Dan Data Aktual
plotdf = pd.DataFrame({'date': close_stock['Tanggal'],
                       'original_close': close_stock['Netto (kg)'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

#Visualisasi Data Dengan Plotly
names = cycle(['Data Aktual','Train predicted Volume','Test predicted Volume'])
fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Volume Sampah(ton)','date': 'Tanggal'})
fig.update_layout(title_text='Plot Data Aktual dan Data Prediksi',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Keterangan')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1413, 1)
Test predicted data:  (1413, 1)


In [44]:
#prediksi Volume Sampah Hari berikutnya
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps))
        
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))
#######

last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

#########

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1)[0].tolist()
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output)).reshape(1,-1)[0].tolist()

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Volume Sampah 30 Hari Sebelumnya','Prediksi Volume Sampah 30 Hari Kedepan'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Volume Sampah(ton)','index': 'Hari'})
fig.update_layout(title_text='Perbandingan Volume Sampah 30 Hari Terakhir dan 30 Hari Kedepan',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Keterangan')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

1 day input [0.46019956 0.05790408 0.71118617 0.72005273 0.50244714 0.28396264
 0.5883468  0.63783239 0.51647509 0.60114257 0.64892205 0.71330587
 0.70956624 0.5147776  0.5239716  0.39396316 0.64806039 0.68130353
 0.57453427 0.46736002 0.56727903 0.62505385 0.44613714 0.8534303
 0.66683613 0.59920382 0.79824047 0.64051218 0.59710997 0.4327043 ]
1 day output [[0.7252592]]
[0.4601995622727351, 0.057904079135574826, 0.711186171954435, 0.7200527340720699, 0.5024471366777535, 0.283962638082272, 0.5883468040739657, 0.6378323883709308, 0.516475089182622, 0.6011425715615145, 0.6489220535268065, 0.7133058748513623, 0.7095662364071897, 0.5147776035293916, 0.5239715994278527, 0.39396315508297863, 0.6480603856825272, 0.6813035311148259, 0.5745342685301671, 0.4673600220586968, 0.5672790252813347, 0.6250538542402676, 0.4461371430540955, 0.8534302996880763, 0.666836128009375, 0.5992038189118859, 0.7982404742619815, 0.6405121753666397, 0.597109966050287, 0.43270429968833923, 0.7252591848373413]
2 day 

In [45]:
#Visualisasi Hasil Prediksi 30 hari dengan Plotly
prediksi = new_pred_plot.iloc[31:].reset_index()
prediksi = prediksi.drop(['index','last_original_days_value'], axis=1)
tanggal = pd.date_range(start='2023-01-01',periods = pred_days)
prediksi['tanggal'] = tanggal
fig = px.bar(prediksi, x='tanggal', y='next_predicted_days_value',
              color='next_predicted_days_value',
             labels={'next_predicted_days_value':'Volume Sampah perhari(ton)'}, height=600)
fig.update_layout(title_text='Prediksi Volume Sampah Pada Bulan Januari 2023',
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.show()

In [46]:
prediksi

,next_predicted_days_value,tanggal
0,51.437065,2023-01-01
1,85.389229,2023-01-02
2,78.396514,2023-01-03
3,65.284933,2023-01-04
4,67.947811,2023-01-05
5,68.918517,2023-01-06
6,69.873880,2023-01-07
7,54.755560,2023-01-08
8,77.279154,2023-01-09
9,72.356070,2023-01-10


In [47]:
prediksi

,next_predicted_days_value,tanggal
0,51.437065,2023-01-01
1,85.389229,2023-01-02
2,78.396514,2023-01-03
3,65.284933,2023-01-04
4,67.947811,2023-01-05
5,68.918517,2023-01-06
6,69.873880,2023-01-07
7,54.755560,2023-01-08
8,77.279154,2023-01-09
9,72.356070,2023-01-10


In [48]:
df = pd.DataFrame(dict(
    Epoch = [100, 300, 500, 1000],
    MSE = [0.018968, 0.018940, 0.018935, 0.018870]
))

df = df.sort_values(by="Epoch")
fig = px.line(df, x="Epoch", y="MSE", title="Pengaruh Nilai Epoch Terhadap MSE",width=800, height=400,markers=True) 
fig.show()

In [49]:
data_perbandingan = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/perbandingan perbulan (fix).xlsx')
data_perbandingan

,Tanggal,Netto (kg)
0,2019-01-03,21.530
1,2019-01-04,52.414
2,2019-01-05,59.410
3,2019-01-06,52.080
4,2019-01-07,85.572
...,...,...
142,2023-01-26,70.632
143,2023-01-27,63.484
144,2023-01-28,62.837
145,2023-01-29,57.613


In [50]:
from datetime import datetime, date 

data_perbandingan["date_added"] = pd.to_datetime(data_perbandingan['Tanggal'])

data_perbandingan['day_added']=data_perbandingan['Tanggal'].dt.day
data_perbandingan['month_added']=data_perbandingan['Tanggal'].dt.month
data_perbandingan['month_name_added']=data_perbandingan['Tanggal'].dt.month_name()
data_perbandingan['year_added'] = data_perbandingan['Tanggal'].dt.year

data_perbandingan.head(3)

,Tanggal,Netto (kg),date_added,day_added,month_added,month_name_added,year_added
0,2019-01-03,21.530,2019-01-03,3,1,January,2019
1,2019-01-04,52.414,2019-01-04,4,1,January,2019
2,2019-01-05,59.410,2019-01-05,5,1,January,2019


In [51]:
#data Sampai Tahun 2022
#sorting data dari 2019-2022
data_januari = data_perbandingan.loc[(data_perbandingan["month_name_added"] == "January")]
data_januari

,Tanggal,Netto (kg),date_added,day_added,month_added,month_name_added,year_added
0,2019-01-03,21.530,2019-01-03,3,1,January,2019
1,2019-01-04,52.414,2019-01-04,4,1,January,2019
2,2019-01-05,59.410,2019-01-05,5,1,January,2019
3,2019-01-06,52.080,2019-01-06,6,1,January,2019
4,2019-01-07,85.572,2019-01-07,7,1,January,2019
...,...,...,...,...,...,...,...
142,2023-01-26,70.632,2023-01-26,26,1,January,2023
143,2023-01-27,63.484,2023-01-27,27,1,January,2023
144,2023-01-28,62.837,2023-01-28,28,1,January,2023
145,2023-01-29,57.613,2023-01-29,29,1,January,2023


In [52]:
fig = px.line(x=data_januari['day_added'], y=data_januari['Netto (kg)'],color=data_januari['year_added'] ,
            labels={
                "y": "Jumlah sampah(ton)",  "x": "Tanggal", "color": "Tahun"
            },title="Jumlah Sampah di Bulan Januari")
fig.show()